# Simulation of the diffraction pattern
This notebook shows how we can create a sample (phase) from atoms and calculate diffraction profile using constant wavelength experiment type.

In [ ]:
# EasyScience, technique-independent
from easyCore import np
from easyCore.Fitting.Fitting import Fitter

# EasyScience, diffraction
from easyDiffractionLib import Site, Phase, Phases
from easyDiffractionLib.Jobs import Powder1DCW

# Vizualization
import py3Dmol
from bokeh.io import show, output_notebook
from bokeh.plotting import figure

# Misc
import xarray as xr

In [ ]:
output_notebook()
FIGURE_WIDTH = 700
FIGURE_HEIGHT = 300

## Sample

### Create an atom using `Site` interface

In [ ]:
atom = Site.from_pars(label="Cl",
                      specie="Cl",
                      fract_x=0.0,
                      fract_y=0.0,
                      fract_z=0.0)

### Create a phase, set space group, add atom

In [ ]:
phase = Phase(name="salt")

phase.spacegroup.space_group_HM_name = "F m -3 m"

phase.add_atom(atom)

### Add another atom (using `Phase` interface)

In [ ]:
phase.add_atom('Na', 'Na', 0.5, 0.5, 0.5)

### Visualise the structure

In [ ]:
viewer = py3Dmol.view()
viewer.addModel(phase.to_cif_str(),'cif',{'doAssembly':True,'duplicateAssemblyAtoms':True,'normalizeAssembly':True})
viewer.setStyle({'sphere':{'colorscheme':'Jmol','scale':.2},'stick':{'colorscheme':'Jmol', 'radius': 0.1}})
viewer.addUnitCell()
viewer.replicateUnitCell(2,2,2)
viewer.zoomTo()

### Create Phases object

In [ ]:
phases = Phases()
phases.append(phase)

## Experiment

### Create `Dataset`

In [ ]:
data = xr.Dataset()

### Create `Job`

In [ ]:
job = Powder1DCW('NaCl', data, phases=phases)

###  Modify instrumental parameters

In [ ]:
job.parameters.resolution_u = 0.1447
job.parameters.resolution_v = -0.4252
job.parameters.resolution_w = 0.3864
job.parameters.resolution_x = 0.0
job.parameters.resolution_y = 0.0

###  Modify pattern parameters

In [ ]:
job.pattern.zero_shift = 0.0
job.pattern.scale = 100.0

###  Create calculator

In [ ]:
calculator = job.interface # CrysPy is default

In [ ]:
print(f"Current calculator engine: {calculator.current_interface_name}")

In [ ]:
print(f"Available calculator engines: {calculator.available_interfaces}")
print(f"Available calculators for CW: {calculator.interface_compatability('Npowder1DCWunp')}")

## Analysis

### Calculate the profile using the calculator we defined previously.

In [ ]:
x_data = np.linspace(20, 170, 500)
_ = job.create_simulation(x_data)
y_data = np.array(data['sim_NaCl'])

In [ ]:
fig = figure(width=FIGURE_WIDTH, height=FIGURE_HEIGHT)
fig.line(x_data, y_data, legend_label='CW Simulation', color='orangered', line_width=2)
show(fig)